In [0]:
from google.colab import drive
drive.mount('/content/drive')
DATA_PATH = "/content/drive/My Drive" 

In [0]:
!pip install contextily
!pip install geopandas
import pandas as pd
import geopandas as gpd
from datetime import datetime, timezone
from shapely.geometry import Point
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
import contextily as ctx
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec

Import data

In [0]:
combined_L_dublin =  pd.read_csv(DATA_PATH + "/dublin/network_combined.csv", sep = ";")
temporal_dublin = pd.read_csv(DATA_PATH + "/dublin/network_temporal_day.csv", sep = ";")
combined_L_adelaide =  pd.read_csv(DATA_PATH + "/adelaide/network_combined.csv", sep = ";")
temporal_adelaide = pd.read_csv(DATA_PATH + "/adelaide/network_temporal_day.csv", sep = ";")
combined_L_rome =  pd.read_csv(DATA_PATH + "/rome/network_combined.csv", sep = ";")
temporal_rome = pd.read_csv(DATA_PATH + "/rome/network_temporal_day.csv", sep = ";")
combined_L_adelaide89 = pd.read_csv(DATA_PATH + "/adelaide/combined_L_8am_9am.csv")
combined_L_adelaide1112 = pd.read_csv(DATA_PATH + "/adelaide/combined_L_11am_12am.csv")
combined_L_dublin89 = pd.read_csv(DATA_PATH + "/dublin/combined_L_8am_9am.csv")
combined_L_dublin1112 = pd.read_csv(DATA_PATH + "/dublin/combined_L_11am_12am.csv")
combined_L_rome89 = pd.read_csv(DATA_PATH + "/rome/combined_L_8am_9am.csv")
combined_L_rome1112 = pd.read_csv(DATA_PATH + "/rome/combined_L_11am_12am.csv")
combined_P_rome1112_sn = pd.read_csv(DATA_PATH + "/rome/descriptive_P1112_rome_list_sn.csv")
combined_P_rome1112_sn_weight = pd.read_csv(DATA_PATH + "/rome/combined1112_P_sn_weight.csv")
combined_P_rome89_sn_weight = pd.read_csv(DATA_PATH + "/rome/combined89_P_sn_weight.csv")
combined_P_adelaide89_sn_weight = pd.read_csv(DATA_PATH + "/adelaide/combined89_P_sn_weight.csv")
combined_P_adelaide1112_sn_weight = pd.read_csv(DATA_PATH + "/adelaide/combined1112_P_sn_weight.csv")
combined_P_dublin89_sn_weight = pd.read_csv(DATA_PATH + "/dublin/combined89_P_sn_weight.csv")
combined_P_dublin1112_sn_weight = pd.read_csv(DATA_PATH + "/dublin/combined1112_P_sn_weight.csv")
combined_L_rome89_graph_sn = nx.read_gpickle(DATA_PATH + "/rome/combined89_L_graph_sn")
combined_L_rome1112_graph_sn = nx.read_gpickle(DATA_PATH + "/rome/combined1112_L_graph_sn")
combined_L_dublin89_graph_sn = nx.read_gpickle(DATA_PATH + "/dublin/combined89_L_graph_sn")
combined_L_dublin1112_graph_sn = nx.read_gpickle(DATA_PATH + "/dublin/combined1112_L_graph_sn")
combined_L_adelaide89_graph_sn = nx.read_gpickle(DATA_PATH + "/adelaide/combined89_L_graph_sn")
combined_L_adelaide1112_graph_sn = nx.read_gpickle(DATA_PATH + "/adelaide/combined1112_L_graph_sn")


rome_1112_gtc = pd.read_csv(DATA_PATH + "/dublin1112_final_gtc.csv", sep=",")
r_routes_geojson = gpd.read_file(DATA_PATH + '/rome/routes.geojson') #Public transport routes in GeoJSON format.
r_sections_geojson = gpd.read_file(DATA_PATH + '/rome/sections.geojson') #Each stop-to-stop section in GeoJSON format.
r_stops_geojson = gpd.read_file(DATA_PATH + '/rome/stops.geojson') #Information on the nodes in GeoJSON format.
a_routes_geojson = gpd.read_file(DATA_PATH + '/adelaide/routes.geojson') #Public transport routes in GeoJSON format.
a_sections_geojson = gpd.read_file(DATA_PATH + '/adelaide/sections.geojson') #Each stop-to-stop section in GeoJSON format.
a_stops_geojson = gpd.read_file(DATA_PATH + '/adelaide/stops.geojson') #Information on the nodes in GeoJSON format.
d_routes_geojson = gpd.read_file(DATA_PATH + '/dublin/routes.geojson') #Public transport routes in GeoJSON format.
d_sections_geojson = gpd.read_file(DATA_PATH + '/dublin/sections.geojson') #Each stop-to-stop section in GeoJSON format.
d_stops_geojson = gpd.read_file(DATA_PATH + '/dublin/stops.geojson') #Information on the nodes in GeoJSON format.

Example: table 1

In [0]:
temporal_dublin

Table 2: Number of connections

In [0]:
temporal_dublin["dep_time_ut"] = pd.to_datetime(temporal_dublin["dep_time_ut"], unit = "s") #transforms Unix timestamp to datetime
temporal_dublin["arr_time_ut"] = pd.to_datetime(temporal_dublin["arr_time_ut"], unit = "s") #ransforms Unix timestamp to  datetime
connections_dublin89 = len(temporal_dublin[(temporal_dublin["dep_time_ut"] >= "2016-12-12 08:00:00") & (temporal_dublin["dep_time_ut"] < "2016-12-12 09:00:00")]) #selecting timespan: 8am-9am
print("dublin 89:",connections_dublin89 )

connections_dublin1112 = len(temporal_dublin[(temporal_dublin["dep_time_ut"] >= "2016-12-12 11:00:00") & (temporal_dublin["dep_time_ut"] < "2016-12-12 12:00:00")]) #selecting timespan: 11am-12am
print("dublin 1112:",connections_dublin1112 )
"""
"""
temporal_adelaide["dep_time_ut"] = pd.to_datetime(temporal_adelaide["dep_time_ut"], unit = "s") #transforms Unix timestamp to datetime
temporal_adelaide["arr_time_ut"] = pd.to_datetime(temporal_adelaide["arr_time_ut"], unit = "s") #ransforms Unix timestamp to  datetime
temporal_adelaide["dep_time_ut"] += pd.to_timedelta(10.5, unit='h') #UTC+10:30h  https://www.timeanddate.com/time/zone/australia/adelaide
temporal_adelaide["arr_time_ut"] += pd.to_timedelta(10.5, unit='h') #UTC+10:30h   https://www.timeanddate.com/time/zone/australia/adelaide
connections_adelaide89 = len(temporal_adelaide[(temporal_adelaide["dep_time_ut"] >= "2016-12-12 08:00:00") & (temporal_adelaide["dep_time_ut"] < "2016-12-12 09:00:00")]) #selecting timespan: 8am-9am
print("adelaide 89:",connections_adelaide89 )

connections_adelaide1112 = len(temporal_adelaide[(temporal_adelaide["dep_time_ut"] >= "2016-12-12 11:00:00") & (temporal_adelaide["dep_time_ut"] < "2016-12-12 12:00:00")]) #selecting timespan: 8am-9am
print("adelaide 1112:",connections_adelaide1112 )
"""
"""
temporal_rome["dep_time_ut"] = pd.to_datetime(temporal_rome["dep_time_ut"], unit = "s") #transforms Unix timestamp to datetime
temporal_rome["arr_time_ut"] = pd.to_datetime(temporal_rome["arr_time_ut"], unit = "s") #ransforms Unix timestamp to  datetime
temporal_rome["dep_time_ut"] += pd.to_timedelta(1, unit='h') #UTC+1h  https://www.timeanddate.com/time/zone/italy/rome
temporal_rome["arr_time_ut"] += pd.to_timedelta(1, unit='h') #UTC+1h  https://www.timeanddate.com/time/zone/italy/rome
connections_rome89 = len(temporal_rome[(temporal_rome["dep_time_ut"] >= "2017-11-06 08:00:00") & (temporal_rome["dep_time_ut"] < "2017-11-06 09:00:00")]) #selecting timespan: 8am-9am
print("rome 89:",connections_rome89 )

connections_rome1112 = len(temporal_rome[(temporal_rome["dep_time_ut"] >= "2017-11-06 11:00:00") & (temporal_rome["dep_time_ut"] < "2017-11-06 12:00:00")]) #selecting timespan: 8am-9am
print("rome 1112:",connections_rome1112 )


Example: table 3

In [0]:
combined_L_rome89.loc[[1,47,43]]

Table 5

In [0]:
combined_P_rome1112.sort_values("from_stop_I")

Table 6

In [0]:
combined_P_rome1112_sn_weight.loc[[143578, 110595, 0, 72067, 118635]]

Table 7

In [0]:
rome_1112_gtc = pd.read_csv(DATA_PATH + "/rome1112_final_gtc.csv", sep=",")
rome_1112_gtc = rome_1112_gtc.set_index("Unnamed: 0")
rome_1112_gtc.index.name = None
rome_1112_gtc

Figure 2: Maps of multimodal public transport system

In [0]:
r_stops_map = r_stops_geojson.to_crs(epsg=3857)
r_routes_map = r_routes_geojson.to_crs(epsg = 3857)
r_sections_map = r_sections_geojson.to_crs(epsg=3857)
r_sections_rail = r_sections_map[r_sections_map["route_type"]==2]
r_sections_tram = r_sections_map[r_sections_map["route_type"]==0]
r_sections_subway = r_sections_map[r_sections_map["route_type"]==1]
r_sections_bus = r_sections_map[r_sections_map["route_type"]==3]

a_stops_map = a_stops_geojson.to_crs(epsg=3857)
a_routes_map = a_routes_geojson.to_crs(epsg = 3857)
a_sections_map = a_sections_geojson.to_crs(epsg=3857)
a_sections_rail = a_sections_map[a_sections_map["route_type"]==2]
a_sections_tram = a_sections_map[a_sections_map["route_type"]==0]
a_sections_bus = a_sections_map[a_sections_map["route_type"]==3]

d_stops_map = d_stops_geojson.to_crs(epsg=3857)
d_routes_map = d_routes_geojson.to_crs(epsg = 3857)
d_sections_map = d_sections_geojson.to_crs(epsg=3857)
d_sections_rail = d_sections_map[d_sections_map["route_type"]==2]
d_sections_tram = d_sections_map[d_sections_map["route_type"]==0]
d_sections_bus = d_sections_map[d_sections_map["route_type"]==3]


fig = fig = plt.figure(figsize = (24, 24))
a_ax = plt.subplot2grid((24, 24), (0, 0), rowspan = 8, colspan= 8)
d_ax = plt.subplot2grid((24, 24), (0, 8), rowspan = 8, colspan= 8)
r_ax = plt.subplot2grid((24, 24), (0, 16), rowspan = 8, colspan= 8)

a_sections_rail.plot(ax=a_ax, label = "rail", zorder = 1, linewidth = 1, color = "red")
a_sections_tram.plot(ax=a_ax, label = "tram", zorder = 3, linewidth = 2, color = "green")
a_sections_bus.plot(ax=a_ax, label = "bus", zorder = 2, linewidth = 1, color = "blue")
a_ax.legend()
a_ax.set_title("Adelaide", size = "x-large", weight = "bold")
a_ax.axis('off')

d_sections_rail.plot(ax=d_ax, label = "rail", zorder = 1, linewidth = 1, color = "red")
d_sections_tram.plot(ax=d_ax, label = "tram", zorder = 3, linewidth = 2, color = "green")
d_sections_bus.plot(ax=d_ax, label = "bus", zorder = 2, linewidth = 1, color = "blue")
d_ax.legend()
d_ax.set_title("Dublin", size = "x-large", weight = "bold")
d_ax.axis('off')

r_sections_rail.plot(ax=r_ax, label = "rail", zorder = 1, linewidth = 1, color = "red")
r_sections_subway.plot(ax=r_ax, label = "subway", zorder = 4, linewidth = 2, color = "orange")
r_sections_tram.plot(ax=r_ax, label = "tram", zorder = 3, linewidth = 2, color = "green")
r_sections_bus.plot(ax=r_ax, label = "bus", zorder = 2, linewidth = 1, color = "blue")
r_ax.legend()
r_ax.set_title("Rome", size = "x-large", weight = "bold")
r_ax.axis('off')

ctx.add_basemap(r_ax)
ctx.add_basemap(d_ax)
ctx.add_basemap(a_ax)




Table 9: Number of nodes per public transport network per network

In [0]:
def count_stops_per_mode(P_space):
  """
  This code computes a list per mode of transport with corresponding stops and 
    a pd.dataframe with the count per mode
  Input: P-space networkx Graph
  Returns: pd.DataFrame with count per mode of transport and a list per mode of transport
  """
  count_tram = 0
  list_tram = []
  count_subway = 0
  list_subway = []
  count_rail = 0
  list_rail = []
  count_bus = 0
  list_bus = []
  for row in P_space.itertuples():
    if row[8] == 0:
      if row[1] not in list_tram:
        list_tram.append(row[1])
        count_tram += 1
      if row[2] not in list_tram:
        list_tram.append(row[2])
        count_tram += 1
    elif row[8] == 1:
      if row[1] not in list_subway:
        list_subway.append(row[1])
        count_subway += 1
      if row[2] not in list_subway:
        list_subway.append(row[2])
        count_subway += 1
    elif row[8] == 2:
      if row[1] not in list_rail:
        list_rail.append(row[1])
        count_rail += 1
      if row[2] not in list_rail:
        list_rail.append(row[2])
        count_rail += 1
    elif row[8] == 3:
      if row[1] not in list_bus:
        list_bus.append(row[1])
        count_bus += 1
      if row[2] not in list_bus:
        list_bus.append(row[2])
        count_bus += 1
  count_df = pd.DataFrame ({"tram_nodes": count_tram, "subway_nodes": count_subway,
                            "rail_nodes": count_rail, "bus_nodes": count_bus}, 
                           index = ["count"])
  return count_df, list_tram, list_subway, list_rail, list_bus
    

In [0]:
combined_P_dublin89_sn_weight = combined_P_dublin89_sn_weight[["source", "target", "frequency", "weight", "waiting_time", "travel_time", "route_I", "route_type"]]
combined_P_dublin1112_sn_weight = combined_P_dublin1112_sn_weight[["source", "target", "frequency", "weight", "waiting_time", "travel_time", "route_I", "route_type"]]
combined_P_adelaide89_sn_weight = combined_P_adelaide89_sn_weight[["source", "target", "frequency", "weight", "waiting_time", "travel_time", "route_I", "route_type"]]
combined_P_adelaide1112_sn_weight = combined_P_adelaide1112_sn_weight[["source", "target", "frequency", "weight", "waiting_time", "travel_time", "route_I",  "route_type"]]

In [0]:
rome89_count_df, rome89_list_tram, rome89_list_subway, rome89_list_rail, rome89_list_bus = count_stops_per_mode(combined_P_rome89_sn_weight)
rome1112_count_df, rome1112_list_tram, rome1112_list_subway, rome1112_list_rail, rome1112_list_bus = count_stops_per_mode(combined_P_rome1112_sn_weight)
dublin89_count_df, dublin89_list_tram, dublin89_list_subway, dublin89_list_rail, dublin89_list_bus = count_stops_per_mode(combined_P_dublin89_sn_weight)
dublin1112_count_df, dublin1112_list_tram, dublin1112_list_subway, dublin1112_list_rail, dublin1112_list_bus = count_stops_per_mode(combined_P_dublin1112_sn_weight)
adelaide89_count_df, adelaide89_list_tram, adelaide89_list_subway, adelaide89_list_rail, adelaide89_list_bus = count_stops_per_mode(combined_P_adelaide89_sn_weight)
adelaide1112_count_df, adelaide1112_list_tram, adelaide1112_list_subway, adelaide1112_list_rail, adelaide1112_list_bus = count_stops_per_mode(combined_P_adelaide1112_sn_weight)

Table 9: Topological features 

In [0]:
rome_89_P_graph = nx.from_pandas_edgelist(combined_P_rome89_sn_weight, source = "source", target = "target", 
            edge_attr = ["route_I", "route_type", "frequency", "travel_time", "waiting_time", "weight"], create_using = nx.MultiGraph())

rome_1112_P_graph = nx.from_pandas_edgelist(combined_P_rome1112_sn_weight, source = "source", target = "target", 
            edge_attr = ["route_I", "route_type", "frequency", "travel_time", "waiting_time", "weight"], create_using = nx.MultiGraph())

dublin_89_P_graph = nx.from_pandas_edgelist(combined_P_dublin89_sn_weight, source = "source", target = "target", 
            edge_attr = ["route_I", "route_type", "frequency", "travel_time", "waiting_time", "weight"], create_using = nx.MultiGraph())
dublin_1112_P_graph = nx.from_pandas_edgelist(combined_P_dublin1112_sn_weight, source = "source", target = "target", 
            edge_attr = ["route_I", "route_type", "frequency", "travel_time", "waiting_time", "weight"], create_using = nx.MultiGraph())

adelaide_89_P_graph = nx.from_pandas_edgelist(combined_P_adelaide89_sn_weight, source = "source", target = "target", 
            edge_attr = ["route_I", "route_type", "frequency", "travel_time", "waiting_time", "weight"], create_using = nx.MultiGraph())
adelaide_1112_P_graph = nx.from_pandas_edgelist(combined_P_adelaide1112_sn_weight, source = "source", target = "target", 
            edge_attr = ["route_I", "route_type", "frequency", "travel_time", "waiting_time", "weight"], create_using = nx.MultiGraph())


In [0]:
def connected_component_subgraphs(G):
  """
  This function selects the connected components in a graph
  """
  for c in nx.connected_components(G):
      yield G.subgraph(c)

def subgraph_check(G):
  """
  This function puts the nodes and edges of the subgraphs into dictionaries.
  Furthermore, this function prints the number of nodes and edges per subgraph
  Input: nx.Graph
  Returns: two dictionaries. First consists of subgraphs and corresponding nodes.
  Second consists of subgraphs and corresponding edges.
  """
  count = 1
  subgraph_nodes_dict = dict()
  subgraph_edges_dict = dict()
  for g in connected_component_subgraphs(G):
    print("number of nodes {}:".format(count), g.number_of_nodes())
    nodes_list = list(g.nodes)
    subgraph_nodes_dict[count] = nodes_list
    print("number of edges {}:".format(count), g.number_of_edges())
    edges_list = list(g.nodes)
    subgraph_edges_dict[count] = edges_list = list(g.nodes)
    count += 1
  return subgraph_nodes_dict, subgraph_edges_dict

In [0]:
#Get nodes and edges in P-space per connected component
print("adelaide 8-9")
adelaide98_P_nodes, adelaide98_P_edges = subgraph_check(adelaide_89_P_graph)
print("adelaide 11-12")
adelaide1112_P_nodes, adelaide98_P_edges = subgraph_check(adelaide_1112_P_graph)
print("dublin 8-9")
dublin98_P_nodes, dublin98_P_edges = subgraph_check(dublin_89_P_graph)
print("dublin 11-12")
dublin1112_P_nodes, dublin1112_P_edges = subgraph_check(dublin_1112_P_graph)
print("rome 8-9")
rome98_P_nodes, rome98_P_edges = subgraph_check(rome_89_P_graph)
print("rome 11-12")
rome1112_P_nodes, rome1112_P_edges = subgraph_check(rome_1112_P_graph)

In [0]:
#Mean degree L-space: k = 2m/n 
print("L-space")
print("Mean degree Adelaide 8-9:", (2*6109)/4776)
print("Mean degree Adelaide 11-12:", (2*5712)/4623)
print("Mean degree Dublin 8-9:", (2*3980)/2937)
print("Mean degree Dublin 11-12:", (2*3823)/2881)
print("Mean degree Rome 8-9:", (2*7723)/5424)
print("Mean degree Rome 11-12:", (2*7694)/5409)
print("P-space")
print("Mean degree Adelaide 8-9:", (2*355162)/4776)
print("Mean degree Adelaide 11-12:", (2*329162)/4623)
print("Mean degree Dublin 8-9:", (2*242313)/2937)
print("Mean degree Dublin 11-12:", (2*230945)/2881)
print("Mean degree Rome 8-9:", (2*223825)/5424)
print("Mean degree Rome 11-12:", (2*222285)/5409)

In [0]:
#Giant connected component
print("GCC Adelaide 8-9:", len(max(nx.connected_components(combined_L_adelaide89_graph_sn), key=len)))
print("GCC Adelaide 11-12:", len(max(nx.connected_components(combined_L_adelaide1112_graph_sn), key=len)))
print("GCC Dublin 8-9:", len(max(nx.connected_components(combined_L_dublin89_graph_sn), key=len)))
print("GCC Dublin 11-12:", len(max(nx.connected_components(combined_L_dublin1112_graph_sn), key=len)))
print("GCC Rome 8-9:", len(max(nx.connected_components(combined_L_rome89_graph_sn), key=len)))
print("GCC Rome 11-12:", len(max(nx.connected_components(combined_L_rome1112_graph_sn), key=len)))

In [0]:
#Average shortest path length
for C in (combined_L_adelaide89_graph_sn.subgraph(c).copy() for c in nx.connected_components(combined_L_adelaide89_graph_sn)):
     print("Average SP Adelaide 8-9:", nx.average_shortest_path_length(C))
     

for C in (combined_L_adelaide1112_graph_sn.subgraph(c).copy() for c in nx.connected_components(combined_L_adelaide1112_graph_sn)):
     print("Average SP Adelaide 11-12:", nx.average_shortest_path_length(C))
    
print("Average SP Dublin 8-9:", nx.average_shortest_path_length(combined_L_dublin89_graph_sn))
print("Average SP Dublin 11-12:", nx.average_shortest_path_length(combined_L_dublin1112_graph_sn))

for C in (combined_L_rome89_graph_sn.subgraph(c).copy() for c in nx.connected_components(combined_L_rome89_graph_sn)):
     print("Average SP Rome 8-9:", nx.average_shortest_path_length(C))

for C in (combined_L_rome1112_graph_sn.subgraph(c).copy() for c in nx.connected_components(combined_L_rome1112_graph_sn)):
     print("Average SP Rome 11-12:", nx.average_shortest_path_length(C))
     

In [0]:
#Random average shortest path length
print("random Adelaide 8-9:", (np.log(4712)-0.5772)/(np.log(2.56)+0.5))
print("random Adelaide 11-12:", (np.log(4559)-0.5772)/(np.log(2.48)+0.5))
print("random Dublin 8-9:", (np.log(2937)-0.5772)/(np.log(2.71)+0.5))
print("random Dublin 11-12:", (np.log(2881)-0.5772)/(np.log(2.65)+0.5))
print("random Rome 8-9:", (np.log(5398)-0.5772)/(np.log(2.85)+0.5))
print("random Rome 11-12:", (np.log(5376)-0.5772)/(np.log(2.85)+0.5))

In [0]:
#Measure of efficiency in terms of path length
print("Adelaide 8-9:", 24.091359347897615/5.472658334783038)
print("Adelaide 11-12:", 33.647889919894794/5.572597818420292)
print("Dublin 8-9:", 23.646986813918815/4.948696139954736)
print("Dublin 11-12:", 30.392837411778316/5.010779174109157)
print("Rome 8-9:", 25.76123603585569/5.180950937208225)
print("Rome 11-12:", 26.094689645625692/5.178311590447757)

In [0]:
#Diameter
for C in (combined_L_adelaide89_graph_sn.subgraph(c).copy() for c in nx.connected_components(combined_L_adelaide89_graph_sn)):
    print("Diameter Adelaide 8-9:", nx.diameter(C))
    
for C in (combined_L_adelaide1112_graph_sn.subgraph(c).copy() for c in nx.connected_components(combined_L_adelaide1112_graph_sn)):
    print("Diameter Adelaide 11-12:", nx.diameter(C))
    
print("Diameter Dublin 8-9:", nx.diameter(combined_L_dublin89_graph_sn))
print("Diameter Dublin 11-12:", nx.diameter(combined_L_dublin1112_graph_sn))

for C in (combined_L_rome89_graph_sn.subgraph(c).copy() for c in nx.connected_components(combined_L_rome89_graph_sn)):
    print("Diameter Rome 8-9:", nx.diameter(C))

for C in (combined_L_rome1112_graph_sn.subgraph(c).copy() for c in nx.connected_components(combined_L_rome1112_graph_sn)):
    print("Diameter Rome 11-12:", nx.diameter(C))